In [16]:
import sounddevice as sd
import numpy as np
import librosa
import speech_recognition as sr
from tensorflow.keras.models import load_model

# Modelin yüklenmesi
model = load_model('modelv3.h5')  # Eğitilmiş modelin dosya yolu

def extract_features(sound_signal, sample_rate):
    sound_signal = sound_signal.astype(np.float32)  # Ensure the audio data is in floating-point format
    mfcc_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=13)
    mfcc_features = librosa.util.normalize(mfcc_features)  # Normalizasyon ekleyin
    pad_width = 44 - mfcc_features.shape[1]
    if pad_width > 0:
        mfccs_scaled_features = np.pad(mfcc_features, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs_scaled_features = mfcc_features[:, :44]  # Gerekiyorsa MFCC'lerin boyutunu kırp
    return mfccs_scaled_features.reshape(1, 44, 13)

# Mikrofondan ses verisi almak için gerekli parametreler
RATE = 44100
DURATION = 5  # Kayıt süresi (saniye)

recognizer = sr.Recognizer()
with sr.Microphone() as source:
    print("Konuşmanızı bekliyorum...")
    audio_data = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio_data, language='tr-TR')
        print("Mikrofon Metin:", text)
        # Ses verisinin özelliklere dönüştürülmesi
        audio_array = np.frombuffer(audio_data.get_raw_data(), np.int16).astype(np.float32)  # Convert audio data to float32
        audio_array /= np.iinfo(np.int16).max  # Normalize to range [-1, 1]
        mfccs_scaled_features = extract_features(audio_array, RATE)
        # Modelin kullanılması ve sonucun alınması
        result_array = model.predict(mfccs_scaled_features)
        result_classes = ['mehmetkocoglu','tahakocer']
        # Tahmin edilen sanatçıları ve olasılıklarını gösterme
        predicted_artist_indices = np.argsort(result_array[0])[::-1]  # Olasılıklara göre sıralanmış tahmin edilen sınıf indisleri
        word_count = len(text.split())
        print("Kelime Sayısı:", word_count)
    except sr.UnknownValueError:
        print("Google Web Speech herhangi bir şey anlamadı")
    except sr.RequestError as e:
        print(f"Google Web Speech hizmetinden sonuç alınamadı; {e}")

print("Tahmin edilen kişiler ve olasılıkları:")
for index in predicted_artist_indices:
    predicted_artist = result_classes[index]
    probability = result_array[0][index]
    print(f"{predicted_artist}: {probability:.2f}")


Konuşmanızı bekliyorum...
Mikrofon Metin: video çekerim Bakayım kanka yapacak bir şey yok modeli yani iyi çekmiyor ağlamışım yazmış
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
Kelime Sayısı: 14
Tahmin edilen kişiler ve olasılıkları:
tahakocer: 0.95
mehmetkocoglu: 0.05
